# Finding Locations

The `arcgis.geoanalytics.find_locations` submodule contains tools to identify areas that meet a number of different criteria you specify. The criteria can be based on attribute queries (for example, parcels that are vacant) and spatial queries (for example, within 1 kilometer of a river). The areas that are found can be selected from existing features (such as existing land parcels), or new features can be created where all the requirements are met.

## Table of Contents
* [Detect Incidents](#Detect-Incidents)
* [Geocode Locations](#Geocode-Locations)
* [Find Dwell Locations](#Find-Dwell-Locations)
* [Find Similar Locations](#Find-Similar-Locations)

This toolset uses distributed processing to complete analytics on your GeoAnalytics Server.

<table>
  <tr>
    <th><center>Tool</center></th>
    <th><center>Description</center></th>    
  </tr>
  <tr>
      <td><a href="https://enterprise.arcgis.com/en/portal/latest/use/geoanalytics-detect-incidents.htm"><p align="left">dtect_incidents</p></a></td>
      <td><p align="left">Creates a layer that detects features that meet a given condition.</p></td>
  </tr>
  <tr>
      <td><a href="https://enterprise.arcgis.com/en/portal/latest/use/geoanalytics-geocode-locations-from-table.htm"><p align="left">geocode_locations</p></a></td>
      <td><p aligh="left">Geocode Locations from Table creates an item in your contents containing the geocoded data.</p></td>
  </tr>
  <tr>
      <td><a href="https://enterprise.arcgis.com/en/portal/latest/use/geoanalytics-find-dwell-locations.htm"><p align="left">find_dwell_locations</p></a></td>
      <td><p align="left">Finds locations where moving objects have stopped, or dwelled, using given time and distance thresholds.</p></td>
  </tr>
  <tr>
      <td><a href="https://enterprise.arcgis.com/en/portal/latest/use/geoanalytics-find-similar-locations.htm"><p align="left">find_similar_locations</p></a></td>
      <td><p align="left">Identifies the candidate features that are most similar or dissimilar to one or more input features based on feature attributes.</p></td>
  </tr>

<b>Necessary imports</b>

In [6]:
# connect to Enterprise GIS
from arcgis.gis import GIS
import arcgis.geoanalytics

portal_gis = GIS("https://pythonapi.playground.esri.com/portal", "arcgis_python", "amazing_arcgis_123")

<b> Get the data</b>

In [10]:
search_result = portal_gis.content.search("bigDataFileShares_all_hurricanes", 
                                          item_type = "big data file share", 
                                          max_items=40)
search_result

[<Item title:"bigDataFileShares_all_hurricanes" type:Big Data File Share owner:api_data_owner>]

In [11]:
data_item = search_result[0]
data_item

<Item title:"bigDataFileShares_all_hurricanes" type:Big Data File Share owner:api_data_owner>

In [12]:
#displays layers in the item
data_item.layers

[<Layer url:"https://pythonapi.playground.esri.com/ga/rest/services/DataStoreCatalogs/bigDataFileShares_all_hurricanes/BigDataCatalogServer/hurricanes">]

In [13]:
hurricanes = data_item.layers[0] #select first layer 
hurricanes

<Layer url:"https://pythonapi.playground.esri.com/ga/rest/services/DataStoreCatalogs/bigDataFileShares_all_hurricanes/BigDataCatalogServer/hurricanes">

In [7]:
search_result = portal_gis.content.get('fcbba38b1efb4f628d7a90e2a50be54d')

In [12]:
table = search_result.tables[0]
table

<Table url:"https://pythonapi.playground.esri.com/server/rest/services/Hosted/ImportantPlaces/FeatureServer/0">

## Detect Incidents

The [`detect_incidents`](https://developers.arcgis.com/rest/services-reference/enterprise/detect-incidents.htm) tool examines time-sequential features using a specified condition. Features that meet the specified condition are marked as incidents. The resulting layer displays the input features in the same format as the input, with additional fields stating if the feature is an incident, the status of the incident, the duration of the incident, and a unique incident identifier.

<center><img src="../../static/img/guide_img/ga/detect_incidents.png" height="300" width="300"></center>

The `detect_incidents` task works with a time-enabled layer of points, lines, areas, or tables that represents an instant in time. Using sequentially ordered features, called tracks, this tool determines which features are incidents of interest. Incidents are determined by conditions that you specify. First, the tool determines which features belong to a track using one or more fields. Using the time at each feature, the tracks are ordered sequentially and the incident condition is applied. Features that meet the starting incident condition are marked as an incident. You can optionally apply an ending incident condition; when the end condition is ‘True’, the feature is no longer an incident. The results will be returned with the original features with new columns representing the incident name and indicate which feature meets the incident condition. You can return all original features, only the features that are incidents, or all of the features within tracks where at least one incident occurred.

For example, suppose you have GPS measurements of hurricanes every 10 minutes. Each GPS measurement records the hurricane’s name, location, time of recording, and wind speed. Using these fields, you could create an incident where any measurement with a wind speed greater than 208 km/h is an incident titled Catastrophic. By not setting an end condition, the incident would end if the feature no longer meets the start condition (wind speed slows down to less than 208).

Using another example, suppose you were monitoring concentrations of a chemical in your local water supply using a field called contanimateLevel. You know that the recommended levels are less than 0.01 mg/L, and dangerous levels are above 0.03 mg/L. To detect incidents, where a value above 0.03mg/L is an incident, and remains an incident until contamination levels are back to normal, you create an incident using a start condition of contanimateLevel > 0.03 and an end condition of contanimateLevel < 0.01. This will mark any sequence where values exceed 0.03mg/L until they return to a value less than 0.01.

In [8]:
from arcgis.geoanalytics.find_locations import detect_incidents

This example finds when and where hurricanes are moving where 

In [17]:
##usage example
incidents_detected = detect_incidents(input_layer=hurricanes, 
                                      track_fields='track_type',
                                      start_condition_expression='$feature["Wind"] < 0.2')

In [18]:
incidents_detected

<Item title:"Detect_Incidents_7RFJIV" type:Feature Layer Collection owner:arcgis_python>

## Geocode Locations

The [`geocode_locations`](https://developers.arcgis.com/rest/services-reference/enterprise/geocode-locations.htm) task geocodes a table from a big data file share. The task uses a geocode utility service configured with your portal. If you do not have a geocode utility service configured, talk to your administrator. [Learn more about configuring a locator service](https://enterprise.arcgis.com/en/portal/latest/administer/windows/configure-portal-to-geocode-addresses.htm).

When preparing to use the Geocode Location task be sure to review [Best Practices for geocoding with GeoAnalytics Server](https://enterprise.arcgis.com/en/portal/latest/use/geoanalytics-geocoding-best-practices.htm).

<center><img src="../../static/img/guide_img/ga/geocode_locations.png" height="300" width="300"></center>

In [2]:
from arcgis.geoanalytics.find_locations import geocode_locations

In [14]:
table.query(as_df=True)

,objectid,place,street,city,state,zip
0,1,Lisa's job,380 New York Street,Redlands,CA,92373
1,2,Mark's job,4511 E Guasti Road,Ontario,CA,91761


In [15]:
geocoded_locs = geocode_locations(input_layer=table, country='CA', output_name='geocoded')
geocoded_locs

2021-06-23 15:23:52,638 arcgis.geoprocessing._tool ERROR Invalid value for parameter inputTable - Details : Accessing URL https://pythonapi.playground.esri.com/server/rest/services/Hosted/ImportantPlaces/FeatureServer/0 failed with error Access to this resource is forbidden, regardless of authorization. (status code 403).. 
2021-06-23 15:23:52,639 arcgis.geoprocessing._tool ERROR Failed.


Exception: Job failed.

## Find Dwell Locations

The [`find_dwell_locations`](https://developers.arcgis.com/rest/services-reference/enterprise/find-dwell-locations.htm) tool determines dwell locations from time-sequential points in a track. Dwell locations are defined as sequential observations with little or no movement over a certain period of time. Depending on the field of application, this may be referred to as stay points or idle detection. Tracks are identified by one or more track fields. The result layer displays the dwell location as points, a convex hull of the dwell locations, or a mean center point of the dwell. The output contains the count of features within a dwell location, the start and end time of the dwell, the duration of the dwell, and any additional statistics that have been calculated. Each track can have 0, 1, or more dwell locations.

<center><img src="../../static/img/guide_img/ga/find_dwell_locations.png" height="300" width="300"></center>

The `find_dwell_locations` works with time-enabled points of type instant to find where points dwell within a specific distance and duration.

Dwell locations are determined using both time (time_tolerance) and distance (distance_tolerance) values. First, the tool assigns features to a track using a unique identifier. Track order is determined by the time of features. Next, the distance between the first observation in a track and the next is calculated. Features are considered to be part of a dwell if two temporally consecutive points stay within the given distance for at least the given duration. When two features are found to be part of a dwell, the first feature in the dwell is used as a reference point, and the tool finds consecutive features that are within the specified distance of the reference point in the dwell. Once all features within the specified distance are found, the tool collects the dwell features and calculates their mean center. Features before and after the current dwell are added to the dwell if they are within the given distance of the dwell location’s mean center. This process continues until the end of the track.

For example, ecologists and conservation workers can use the Find Dwell Locations tool to improve the safety of elk during migratory seasons. Leverage the results to implement or improve protected areas in locations where the animals are spending the most time.

For another example, let’s say you work with the Department of Transportation and you want to improve traffic congestion on highways near exits. Using the Find Dwell Locations tool, you can isolate areas experiencing congestion by identifying vehicle tracks that stay within a certain distance for a certain amount of time.

In [19]:
from arcgis.geoanalytics.find_locations import find_dwell_locations

In [21]:
dwell_locs = find_dwell_locations(input_layer=hurricanes,
                                  track_fields='track_type',
                                  distance_tolerance=1,
                                  distance_unit='Meters',
                                  time_tolerance='1',
                                  time_unit='Hours', 
                                  output_name='dwell locations')

In [23]:
dwell_locs

<Item title:"dwell_locations" type:Feature Layer Collection owner:arcgis_python>

## Find Similar Locations

The `find_similar_locations` tool identifies the candidate features that are most similar or dissimilar to one or more input features based on feature attributes.

<center><img src="../../static/img/guide_img/ga/find_similar_locations.png" height="300" width="300"></center>

The find_similar_locations task measures the similarity of candidate locations to one or more reference locations.

Based on criteria you specify, find_similar_locations can answer questions such as the following:
- Which of your stores are most similar to your top performers with regard to customer profiles?
- Based on characteristics of villages hardest hit by the disease, which other villages are high risk?
- To answer questions such as these, you provide the reference locations (the input_layer parameter), the candidate locations (the search_layer parameter), and the fields representing the criteria you want to match. For example, the input_layer might be a layer containing your top performing stores or the villages hardest hit by the disease. The search_layer contains your candidate locations to search. This might be all of your stores or all other villages. Finally, you supply a list of fields to use for measuring similarity. The find_similar_locations task will rank all of the candidate locations by how closely they match your reference locations across all of the fields you have selected.

In [2]:
from arcgis.geoanalytics.find_locations import find_similar_locations 

In [3]:
search_result = portal_gis.content.search("", 
                                          item_type = "big data file share", 
                                          max_items=40)
search_result

[<Item title:"bigDataFileShares_hurricanes_dask_shp" type:Big Data File Share owner:atma.mani>,
 <Item title:"bigDataFileShares_NYC_taxi_data15" type:Big Data File Share owner:api_data_owner>,
 <Item title:"bigDataFileShares_hurricanes_dask_csv" type:Big Data File Share owner:atma.mani>,
 <Item title:"bigDataFileShares_all_hurricanes" type:Big Data File Share owner:api_data_owner>,
 <Item title:"bigDataFileShares_ServiceCallsOrleans" type:Big Data File Share owner:portaladmin>,
 <Item title:"bigDataFileShares_ServiceCallsOrleansTest" type:Big Data File Share owner:arcgis_python>,
 <Item title:"bigDataFileShares_GA_Data" type:Big Data File Share owner:arcgis_python>,
 <Item title:"bigDataFileShares_GA_Data" type:Big Data File Share owner:arcgis_python>,
 <Item title:"bigDataFileShares_Chicago_Crimes" type:Big Data File Share owner:arcgis_python>]

In [6]:
layers = search_result[-1].layers
layers

[<Layer url:"https://pythonapi.playground.esri.com/ga/rest/services/DataStoreCatalogs/bigDataFileShares_Chicago_Crimes/BigDataCatalogServer/crime">,
 <Layer url:"https://pythonapi.playground.esri.com/ga/rest/services/DataStoreCatalogs/bigDataFileShares_Chicago_Crimes/BigDataCatalogServer/homicides">]

In [7]:
crime_incidents = layers[0]
homicides = layers[1]

In [12]:
similar_locs = find_similar_locations(input_layer=crime_incidents,
                                      search_layer=homicides,
                                      analysis_fields='Beat',
                                      most_or_least_similar='MostSimilar',
                                      match_method='AttributeValues',
                                      number_of_results=10,
                                      return_tuple=True,
                                      output_name='similar locations')

In [16]:
similar_locs.output

<FeatureLayer url:"https://pythonapi.playground.esri.com/server/rest/services/Hosted/similar_locations/FeatureServer/0">